- join tables
- remove duplicates
- outliers / missing data
- standarize names -> coordinates to names of places
- ...

deixo aixo aqui per si pot servir despres

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

from pathlib import Path

In [2]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'app_adsdb'

#geolocator = Nominatim(user_agent='app_adsdb/1')
geolocator = Nominatim(timeout=3)

In [3]:

def postcode(df,latitude="Latitude",longitude="Longitude"):
    Lat = np.array(df[[latitude]]).flatten()
    Long = np.array(df[[longitude]]).flatten()

    coordinates = [(Lat[i],Long[i]) for i in range(len(Lat))]#len(Lat)
    location = [geolocator.reverse(coor).raw["address"] for coor in coordinates]

    postcode = []

    for loc in location:
        if "postcode" in loc:
            postcode.append(loc["postcode"])
        else:
            postcode.append("")

    return df.assign(ZIP=list(postcode))
    


In [4]:
def drop_columns(df,alpha=0.9, constants=True):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    missings = 0
    n = len(df)
    drop_list = []

    for i in range(len(columns_names)):

        if types_df[i] == "float64" or types_df[i] == "int32":
            missings = df[columns_names[i]].isna().sum()
            
            if constants and min(df[columns_names[i]]) == max(df[columns_names[i]]):
                drop_list.append(columns_names[i])
                continue

        elif types_df[i] == "object":
            counts = dict(df[columns_names[i]].value_counts())
            if "unknow" in counts:
                missings = counts["unknow"]

            if constants and len(counts) == 1:
                drop_list.append(columns_names[i])
                continue

        if missings/n > alpha:
            drop_list.append(columns_names[i])

    return df.drop(drop_list,axis=1)


In [5]:
# numeric -> np.nan
# object -> "unknow"
def standar_missings(df):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    
    for i in range(len(columns_names)):
        
        if types_df[i] == "float64" or types_df[i] == "int32":
            df[columns_names[i]] = df[columns_names[i]].replace(-999, np.nan)

        elif types_df[i] == "object":
            df[columns_names[i]] = df[columns_names[i]].replace(np.nan, "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("xxxxxx", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("nan", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("......", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("NOT AVAILABLE", "unknow")
            

In [1]:
def formatted2trusted():
    
    # list of tables already processed:
    processed_table = []
    with open('logs/processed_data_trusted.txt', "r") as f0:
        for line in f0:
            processed_table.append(line.rstrip())
    
    # process datasets:
    for datasource in Objects:
        id = datasource["id"]

        # conect to DB
        con = duckdb.connect(database="data/formatted/db_{}.db".format(id), read_only=True)

        # take all tables names
        tables = con.execute("SHOW TABLES").fetchall()
        tables_names = [x[0] for x in tables]

        # tables to process:
        tables_not_processed = [table for table in tables_names if not table in processed_table]
        if len(tables_not_processed) == 0:
            continue
        
        # list of all tables in dataframe format
        dataframes = []
        for table in tables_not_processed:
            dataframes.append(con.execute("SELECT * FROM {}".format(table)).fetchdf())

        # close database
        con.close()

        # merge all dataframes
        df = pd.concat(dataframes)
        standar_missings(df)
        df.drop_duplicates()
        df = drop_columns(df)

        my_file = Path("data/trusted/db_{}.db".format(id))
        if my_file.is_file():
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            df_original = con.execute("SELECT * FROM {}".format(id)).fetchdf()
            df = pd.concat([df,df_original])
            df.drop_duplicates()#CREATE OR REPLACE TABLE
            df = drop_columns(df)
            con.execute("CREATE OR REPLACE TABLE {} AS SELECT * FROM df".format(id))
            con.close()
        
        else:
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            con.execute("CREATE TABLE {} AS SELECT * FROM df".format(id))
            con.close()

        if os.path.exists("scripts/trusted_{}.py".format(id)):
            os.system("python3 ./scripts/trusted_{}.py".format(id))

        with open('logs/processed_data_trusted.txt', "a") as f0:
            for table in tables_not_processed:
                f0.write(table + "\n")

    

In [2]:
formatted2trusted()

FileNotFoundError: [Errno 2] No such file or directory: 'logs/processed_data_trusted.txt'

In [ ]:
con = duckdb.connect(database="data/trusted/db_schools.db", read_only=False)
df = con.execute("SELECT * FROM schools;").fetchdf()
con.close()
df[["LEVEL_","TYPE","STATUS"]].head(10)

,LEVEL_,TYPE,STATUS
0,preschool,Type 1,STATUS 1
1,elementary_school,Type 1,STATUS 1
2,elementary_school,Type 1,STATUS 1
3,preschool,Type 1,STATUS 1
4,elementary_school,Type 1,STATUS 1
5,preschool,Type 1,STATUS 1
6,elementary_school,Type 1,STATUS 1
7,unknown,Type 1,STATUS 2
8,elementary_school,Type 1,STATUS 1
9,middle_school,Type 1,STATUS 1


In [ ]:
con = duckdb.connect(database="data/trusted/db_housing.db", read_only=False)
df = con.execute("SELECT * FROM housing;").fetchdf()
con.close()
df[["cats_allowed","dogs_allowed","smoking_allowed","wheelchair_access","comes_furnished"]].head(10)

,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,comes_furnished
0,Yes,Yes,Yes,No,No
1,Yes,Yes,Yes,No,Yes
2,Yes,Yes,Yes,No,No
3,Yes,Yes,Yes,No,No
4,Yes,Yes,Yes,No,No
5,Yes,Yes,Yes,No,No
6,Yes,Yes,Yes,No,No
7,Yes,Yes,Yes,No,No
8,Yes,Yes,No,No,No
9,Yes,Yes,Yes,No,No


In [ ]:
df2 = df[df["price"] < 100]
numeric_description(df2)

,mean,std,min,25%,50%,75%,max,missings,missing ratio (%)
id,7.039424e+09,9.657351e+06,7.011970e+09,7.032155e+09,7.040225e+09,7.048582e+09,7.051061e+09,0.0,0.00
price,4.175294e+01,3.065533e+01,5.000000e+00,2.225000e+01,2.500000e+01,5.900000e+01,9.900000e+01,0.0,0.00
sqfeet,1.356592e+03,1.521775e+03,8.000000e+00,6.500000e+02,8.150000e+02,1.070000e+03,5.550000e+03,1.0,0.59
beds,1.958824e+00,1.368954e+00,0.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,6.000000e+00,0.0,0.00
baths,1.411765e+00,1.077408e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.500000e+00,0.0,0.00
electric_vehicle_charge,1.176471e-02,1.081438e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,0.00
lat,3.934075e+01,5.589258e+00,2.746350e+01,3.460035e+01,4.061985e+01,4.302980e+01,6.119610e+01,0.0,0.00
long,-9.059822e+01,1.281657e+01,-1.499130e+02,-9.791570e+01,-8.438590e+01,-8.375930e+01,-7.141610e+01,0.0,0.00


In [ ]:
id = "schools"
con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=True)
df = con.execute("SELECT * FROM {}".format(id)).fetchdf()
#df.describe()
df.head()


,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,-89.542799,32.728496,2005,280252001118,LEAKE CENTRAL ELEMENTARY SCHOOL,603 HWY. 16 WEST,CARTHAGE,MS,39051,unknow,...,IMAGERY/OTHER,2010-07-06,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,1159,PK,05,2802520,63,unknow
1,-96.657221,40.764352,2007,317284001170,POUND MIDDLE SCHOOL,4740 S 45TH ST,LINCOLN,NE,68516,1799,...,IMAGERY/OTHER,2010-08-23,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,elementary_school,827,06,08,3172840,53,10786596
2,-115.212889,36.007521,2009,320006000700,LOIS & JERRY TARKANIAN MIDDLE SCHOOL,5800 W. PYLE AVE.,LAS VEGAS,NV,89141,unknow,...,IMAGERY,2010-05-21,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,elementary_school,1739,06,08,3200060,71,unknow
3,-74.448768,39.375791,2010,340096000030,VENICE PARK SCHOOL,1601 N PENROSE AVENUE,ATLANTIC CITY,NJ,8401,1710,...,IMAGERY/OTHER,2010-06-28,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,90,PK,PK,3400960,10,unknow
4,-106.614674,31.835316,2012,350108000660,SANTA TERESA MIDDLE,4800 MCNUTT RD,SANTA TERESA,NM,88008,unknow,...,IMAGERY,2010-05-25,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,elementary_school,605,07,08,3501080,43,unknow


In [ ]:
import re
Col = sorted(df['COUNTY'].apply(lambda x: str(x)).unique())
print(len(Col))
"""
for i in Col:
    if not re.match("[0123456789]*", i) :
        print(i)

"""
Col.append("unknow")
Col2 = [x for x in Col if len(x) < 15]
print(len(Col2))
print("unknow" in Col2)
Col2


1908
1889
True


['ABBEVILLE',
 'ACADIA',
 'ACCOMACK',
 'ADA',
 'ADAIR',
 'ADAMS',
 'ADDISON',
 'ADJUNTAS',
 'AGUADA',
 'AGUADILLA',
 'AGUAS BUENAS',
 'AIBONITO',
 'AIKEN',
 'AITKIN',
 'ALACHUA',
 'ALAMANCE',
 'ALAMEDA',
 'ALAMOSA',
 'ALBANY',
 'ALBEMARLE',
 'ALCONA',
 'ALCORN',
 'ALEUTIANS EAST',
 'ALEUTIANS WEST',
 'ALEXANDER',
 'ALEXANDRIA',
 'ALFALFA',
 'ALGER',
 'ALLAMAKEE',
 'ALLEGAN',
 'ALLEGANY',
 'ALLEGHANY',
 'ALLEGHENY',
 'ALLEN',
 'ALLENDALE',
 'ALPENA',
 'ALPINE',
 'AMADOR',
 'AMELIA',
 'AMHERST',
 'AMITE',
 'ANCHORAGE',
 'ANDERSON',
 'ANDREW',
 'ANDREWS',
 'ANDROSCOGGIN',
 'ANGELINA',
 'ANNE ARUNDEL',
 'ANOKA',
 'ANSON',
 'ANTELOPE',
 'ANTRIM',
 'APACHE',
 'APPANOOSE',
 'APPLING',
 'APPOMATTOX',
 'ARANSAS',
 'ARAPAHOE',
 'ARCHER',
 'ARCHULETA',
 'ARECIBO',
 'ARENAC',
 'ARKANSAS',
 'ARLINGTON',
 'ARMSTRONG',
 'AROOSTOOK',
 'ARROYO',
 'ARTHUR',
 'ASCENSION',
 'ASHE',
 'ASHLAND',
 'ASHLEY',
 'ASHTABULA',
 'ASOTIN',
 'ASSUMPTION',
 'ATASCOSA',
 'ATCHISON',
 'ATHENS',
 'ATKINSON',
 'ATLANTIC',